### Importing the libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Read the data

In [4]:
data1=pd.read_csv('survey_results_public.csv')
data1.head()

FileNotFoundError: [Errno 2] No such file or directory: 'survey_results_public.csv'

In [ ]:
data2=pd.read_csv('survey_results_schema.csv')
data2.head()

In [ ]:
data1.shape

In [ ]:
data2.shape

In [ ]:
data1.columns

### Checking null values

In [ ]:
data1.isnull().sum()

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
data1.head()

In [ ]:
data1['Age1stCode'].isnull().sum()

In [ ]:
data1.describe()

### Q.1 Find the average age of developers when they wrote their first line of code.

In [ ]:
pd.set_option('display.max_rows',None)

In [ ]:
data1['Age1stCode'].unique()

In [ ]:
mask=data1['Age1stCode']=='Older than 85'
data1.loc[mask,('Age1stCode')]=86

In [ ]:
mask=data1['Age1stCode']=='Younger than 5 years'
data1.loc[mask,('Age1stCode')]=4

In [ ]:
data1['Age1stCode'].unique()

In [ ]:
data1['Age1stCode'].isnull().sum()

In [ ]:
data1['Age1stCode']=data1['Age1stCode'].astype('float')

In [ ]:
plt.figure(figsize=(16,5))
sns.countplot(x='Age1stCode',data=data1,order=data1['Age1stCode'].value_counts().iloc[:20].index)

In [ ]:
np.mean(data1['Age1stCode'])

### Average age of developers when they wrote their first line of code is 15.

### Q.2. Deduce the percentage of developers who know python in each country. 

In [ ]:
data1['Country'].nunique()

In [ ]:
languages=data1[data1['LanguageWorkedWith'].notnull()]

In [ ]:
languages.head()

In [ ]:
countries=languages['Country'].unique()

In [ ]:
languages.groupby(['Country'])['LanguageWorkedWith'].value_counts()

In [ ]:
country_python={}

In [ ]:
for country in countries:
    for i in languages.groupby(['Country'])['LanguageWorkedWith'].value_counts()[country].index:
        l=[]
        l.append(i.split(';'))
        for j in l:
            if 'Python' in j:
                if country not in country_python.keys():
                    country_python[country]=1
                else:
                    country_python[country]+=1

In [ ]:
country_python

In [ ]:
def plot_dimension_count(unique_dim_dict,plot_title):
    dim_count=pd.DataFrame.from_dict(unique_dim_dict,orient='index',dtype=None)
    dim_count.columns=['Count']
    dim_count.sort_values('Count',ascending=False,inplace=True)
    dim_count[:10].plot(kind='pie',figsize=(12,12),fontsize=10,title=plot_title,subplots=True,autopct='%1.1f%%');
    print(dim_count[:10])

In [ ]:
plot_dimension_count(country_python,'The most popular languages')

### Q.3. Generate a report for the average salary of developer based on continent. 

In [ ]:
import pycountry_convert as pc

In [ ]:
def country_to_continent(country_name):
    if country_name!='Other Country (Not Listed Above)':
        try:
            country_alpha2=pc.country_name_to_country_alpha2(country_name)
            country_continent_code=pc.country_alpha2_to_continent_code(country_alpha2)
            country_continent_name=pc.convert_continent_code_to_continent_name(country_continent_code)
            return country_continent_name
        except:
            return "unknown"
    else:
        return "unknown"

In [ ]:
data1['Country'].replace({'Hong Kong (S.A.R.)':'Hong Kong'},inplace=True)
data1['Country'].replace({'Libyan Arab Jamahiriya':'Libya'},inplace=True)
data1['Country'].replace({'Venezuela, Bolivarian Republic of...':'Venezuela'},inplace=True)
data1['Country'].replace({'The former Yugoslav Republic of Macedonia':'North Macedonia'},inplace=True)
data1['Country'].replace({'Republic of Korea':'South Korea'},inplace=True)
data1['Country'].replace({'Congo, Republic of the...':'Republic of the Congo'},inplace=True)
data1['Country'].replace({'Timor-Leste':'East Timor'},inplace=True)

In [ ]:
names=[country_to_continent(i) for i in data1['Country']]
data1['Continent']=names

In [ ]:
salary_notnull=data1[data1['ConvertedComp'].notnull()]

In [ ]:
a=salary_notnull.groupby(['Continent'])['ConvertedComp'].mean().sort_values(ascending=False)

In [ ]:
a.index

In [ ]:
plt.figure(figsize=(16,9))
plt.title('Salary countplot based on Continent',fontsize=20,fontweight='bold',y=1.05,)
plt.ylabel('Salary',fontsize=15)
plt.xlabel('Continent',fontsize=15)
a.plot(kind='bar')

### Q.4.  Based on this survey, what will be the most desired programming language for the year 2020? 

In [ ]:
language_desired=data1[data1['LanguageDesireNextYear'].notnull()]

In [ ]:
unique_lang={}

In [ ]:
for language_set in languages['LanguageDesireNextYear'].apply(lambda row: str(row).split(';')):
    for language in language_set:
        if language not in unique_lang.keys():
            unique_lang[language]=1
        else:
            unique_lang[language]+=1

In [ ]:
unique_lang

In [ ]:
def plot_dimension_count(unique_dim_dict,plot_title):
    dim_count=pd.DataFrame.from_dict(unique_dim_dict,orient='index',dtype=None)
    dim_count.columns=['Count']
    dim_count.sort_values('Count',ascending=True,inplace=True)
    dim_count.plot(kind='barh',figsize=(12,12),fontsize=10,title=plot_title);

In [ ]:
plot_dimension_count(unique_lang,'The most popular Languages')

### Q.5. What is the distribution of people who code as a hobby based on gender and continent (hint: use your mapping of country to continent)? 

In [ ]:
data1['Gender'].unique()

In [ ]:
df=data1[data1['Hobbyist']=='Yes']

In [ ]:
data1['Hobbyist']=data1['Hobbyist'].map({'Yes':1,'No':0})

In [ ]:
data1['Gender'].value_counts()

In [ ]:
l=[]
for i in data1['Gender']:
    if i != 'Man' and i != 'Woman':
        l.append('others')
    else:
        l.append(i)

In [ ]:
data1['Gender']=l

In [ ]:
data1['Gender'].value_counts()

In [ ]:
plt.figure(figsize=(12,7))
sns.barplot(x='Gender',y='Hobbyist',hue='Continent',data=data1)
plt.grid()
N=3
ind=np.arange(N)
s=(data1['Gender'].value_counts().index)
plt.xticks(ind,s,rotation='vertical',fontsize=15)
plt.show()

### Q.6. Generate the report for job and career satisfaction of developer based on their gender and continent? 

In [ ]:
data1['JobSat'].value_counts()

In [ ]:
data1['JobSat']=data1['JobSat'].map({'Slightly satisfied':5,'Very satisfied':4, 'Slightly dissatisfied':2, 'Neither satisfied nor dissatisfied':3, 'Very dissatisfied':1})

In [ ]:
plt.figure(figsize=(12,7))
sns.barplot(x='Gender',y='JobSat',hue='Continent',data=data1)
plt.grid()
N=3
ind=np.arange(N)
s=(data1['Gender'].value_counts().index)
plt.xticks(ind,s,rotation='vertical',fontsize=15)
plt.show()